In [17]:
%cd '/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect'

/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect


In [18]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from augment.final.face_org import * 

RuntimeError: Unable to open augment/parsing/shape_predictor_68_face_landmarks.dat

In [30]:

train_images = []
test_images = []
train_labels = []
test_labels = []
sick = 1
healthy = 0

image_folder = '/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect/data/Original'
files = os.listdir(image_folder)

for f in files:
    full_path = image_folder + "/" + str(f)
    image = readAndResize(full_path)
    # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # plt.figure()
    if "n-" in f:
        train_images.append(image)
        train_labels.append(healthy)
    elif "s-" in f:
        train_images.append(image)
        train_labels.append(sick)
    else:       
        test_images.append(image)
        test_labels.append(healthy)

print('train_images shape:', np.array(train_images).shape)
print('test_images shape:', np.array(test_images).shape)

train_images = np.asarray(train_images)
test_images = np.asarray(test_images)
train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)

#  Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 215.0, test_images / 215.0

train_images shape: (52, 512, 512, 3)
test_images shape: (22, 512, 512, 3)


In [ ]:
### Verify the data

To verify that the dataset looks correct, let's plot the first 25 images from the training set and display the class name below each image.


In [ ]:
class_names = ['healthy', 'sick']

plt.figure(figsize=(10,10))
for i in range(5):
    plt.subplot(2,2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    # plt.xlabel(class_names[train_labels[i]])
plt.show()

### Create the convolutional base

The 6 lines of code below define the convolutional base using a common pattern: a stack of [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) and [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) layers.

As input, a CNN takes tensors of shape (image_height, image_width, color_channels), ignoring the batch size. If you are new to these dimensions, color_channels refers to (R,G,B). In this example, you will configure our CNN to process inputs of shape (32, 32, 3), which is the format of CIFAR images. You can do this by passing the argument `input_shape` to our first layer.


In [36]:
model = models.Sequential()
model.add(layers.Conv2D(512, (3, 3), activation='relu', input_shape=(512, 512, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

Let's display the architecture of our model so far.

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 510, 510, 512)     14336     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 255, 255, 512)     0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 253, 253, 512)     2359808   
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 126, 126, 512)     0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 124, 124, 512)     2359808   
Total params: 4,733,952
Trainable params: 4,733,952
Non-trainable params: 0
_________________________________________________________________


Above, you can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as you go deeper in the network. The number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64). Typically,  as the width and height shrink, you can afford (computationally) to add more output channels in each Conv2D layer.

### Add Dense layers on top
To complete our model, you will feed the last output tensor from the convolutional base (of shape (4, 4, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, you will flatten (or unroll) the 3D output to 1D,  then add one or more Dense layers on top. CIFAR has 10 output classes, so you use a final Dense layer with 10 outputs and a softmax activation.

In [38]:
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(2))

ResourceExhaustedError: OOM when allocating tensor with shape[3936256,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:Mul] name: dense_10/kernel/Initializer/random_uniform/mul/

Here's the complete architecture of our model.

In [ ]:
model.summary()

As you can see, our (4, 4, 64) outputs were flattened into vectors of shape (1024) before going through two Dense layers.

### Compile and train the model

In [23]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

ValueError: Error when checking input: expected conv2d_input to have shape (32, 32, 3) but got array with shape (512, 512, 3)

### Evaluate the model

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)

Our simple CNN has achieved a test accuracy of over 70%. Not bad for a few lines of code! For another CNN style, see an example using the Keras subclassing API and a `tf.GradientTape` [here](https://www.tensorflow.org/tutorials/quickstart/advanced).